### Tous les choix dans ce code se base sur notre fouille de donnée

In [1]:
import pandas as pd
import numpy as np
import networkx as nx

from sklearn.cluster import KMeans
from itertools import chain
from itertools import combinations

In [ ]:
# Charger les données des utilisateurs et des recettes depuis des fichiers CSV
# https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions
usr_rates = pd.read_csv('data/RAW_interactions.csv').copy()
rcp = pd.read_csv('data/RAW_recipes.csv').copy()

On cherche à séparer les plats des desserts, plus précisemment le salé du sucré. Comme notre algorithme se base sur la distance entre les utilisateurs pour recommander un plat ou un dessert, il nous semble pertinent de les séparer.\
Pour les séparer, on séléctionne les tags les plus récurrents dans le dataset :\
meat, vegetables et seafood pour les plats\
desserts, cookies-and-brownies et chocolate pour les desserts

In [3]:
# Filtrer les recettes par plat et dessert en fonction des tags choisit
rcp_main_dish = rcp[rcp['tags'].apply(lambda x: 'main-dish' in x or 'meat' in x or 'vegetables' in x or 'seafood' in x)]
rcp_dessert = rcp[rcp['tags'].apply(lambda x: 'desserts' in x or 'cookies-and-brownies' in x or 'chocolate' in x)]

In [4]:
# Créer des ensembles (sets) d'identifiants de recettes pour les plats principaux et les desserts
main_dish_ids = set(rcp_main_dish['id'])
dessert_ids = set(rcp_dessert['id'])

# Définir une fonction pour déterminer le type de plat (main dish ou dessert) en fonction de l'ID de la recette
def get_type_of_dish(recipe_id):
    if recipe_id in dessert_ids:
        return 'dessert' # Si l'ID appartient aux desserts
    elif recipe_id in main_dish_ids:
        return 'main' # Si l'ID appartient aux plats principaux
    else:
        return 'none' # Si l'ID n'appartient à aucun des deux

# Appliquer la fonction pour ajouter une colonne 'type_of_dish' dans les données des recettes
rcp['type_of_dish'] = rcp['id'].apply(get_type_of_dish)

# Filtrer les recettes pour garder uniquement celles classées comme dessert ou plat principal
rcp = rcp[rcp['type_of_dish'] != 'none']

Les review et date ne sont pas importante 

On récupère dans des listes toutes les notes de chaque utilisateur pour binariser leur notes entre -1 et 1 (j'aime pas, j'aime) de façon personelle

In [5]:
# Supprimer les colonnes inutiles des interactions utilisateur
usr_rates.drop(columns=['review', 'date'], inplace=True)

# Regrouper les interactions par utilisateur et agréger les valeurs sous forme de listes
usr_rates = usr_rates.groupby('user_id').aggregate(list)

In [6]:
# Mettre à jour les ensembles d'identifiants pour les desserts et les plats principaux
dessert_ids = set(rcp[rcp['type_of_dish'] == 'dessert']['id'])
main_dish_ids = set(rcp[rcp['type_of_dish'] == 'main']['id'])

# Fonction pour extraire les desserts depuis une liste d'IDs de recettes
def get_list_of_desserts(recipe_id):
    x = []
    for i in recipe_id:
        if i in dessert_ids:
            x.append(i)
    return x

# Fonction pour extraire les plats principaux depuis une liste d'IDs de recettes
def get_list_of_mains(recipe_id):
    x = []
    for i in recipe_id:
        if i in main_dish_ids:
            x.append(i)
    return x

# Appliquer les fonctions pour ajouter des colonnes 'recipes_dessert' et 'recipes_main' dans les données utilisateur
usr_rates['recipes_dessert'] = usr_rates['recipe_id'].apply(get_list_of_desserts)
usr_rates['recipes_main'] = usr_rates['recipe_id'].apply(get_list_of_mains)

Binarisation des notes via un Kmeans

In [7]:
# Fonction pour binariser les notes en utilisant K-Means
def cluster_ratings(ratings):
    km = KMeans(n_clusters=2, verbose=0, random_state=42)
    ratings_array = np.array(ratings).reshape(-1, 1)
    if len(set(ratings)) > 1:
        km.fit(ratings_array)
        
        # Calculer les moyennes des clusters
        cluster_0_mean = np.mean(ratings_array[km.labels_ == 0])
        cluster_1_mean = np.mean(ratings_array[km.labels_ == 1])
        
        # Identifier le cluster avec la moyenne la plus élevée
        high_label = 0 if cluster_0_mean > cluster_1_mean else 1
        
        # Assigner 1 aux notes les plus hautes et -1 aux autres
        return [1 if label == high_label else -1 for label in km.labels_]
    else:
        # Si toutes les notes sont identiques, les considérer comme "likées"
        return [1 for _ in ratings]

# Ajouter une colonne 'ratings_binary' avec les notes binarisées
usr_rates['ratings_binary'] = usr_rates['rating'].apply(cluster_ratings)

In [8]:
# Initialiser deux nouvelles colonnes pour les notes binaires des desserts et des plats principaux
usr_rates['ratings_dessert_binary'] = usr_rates['rating'].apply(lambda x: [])
usr_rates['ratings_main_binary'] = usr_rates['rating'].apply(lambda x: [])

In [9]:
# Remplir les colonnes des notes binaires pour les desserts et les plats principaux
for i in usr_rates.index: # Parcourir chaque utilisateur
    
    # Si l'utilisateur a noté des desserts
    if usr_rates['recipes_dessert'][i] != []:

        # Associer la note binaire à la recette de dessert correspondante
        for j in range(len(usr_rates['recipes_dessert'][i])):
            usr_rates['ratings_dessert_binary'][i].append(usr_rates['ratings_binary'][i][usr_rates['recipe_id'][i].index(usr_rates['recipes_dessert'][i][j])])

    # Si l'utilisateur a noté des plats principaux
    if usr_rates['recipes_main'][i] != []:
        
        # Associer la note binaire à la recette de plat principal correspondante
        for j in range(len(usr_rates['recipes_main'][i])):
            usr_rates['ratings_main_binary'][i].append(usr_rates['ratings_binary'][i][usr_rates['recipe_id'][i].index(usr_rates['recipes_main'][i][j])])

Séparation des plats et desserts dans deux dataframe différents

In [10]:
# Extraire uniquement les colonnes des recettes principales et leurs notes binaires
usr_rates_main_dish = usr_rates[['recipes_main', 'ratings_main_binary']]

# Renommer les colonnes pour qu'elles soient plus génériques et adaptées à l'utilisation ultérieure
usr_rates_main_dish.columns = ['recipes_id', 'rates']

# Réinitialiser l'index pour transformer l'index en colonne et obtenir une structure tabulaire standard
usr_rates_main_dish.reset_index(drop=False, inplace=True)

# Extraire uniquement les colonnes des desserts et leurs notes binaires
usr_rates_dessert = usr_rates[['recipes_dessert', 'ratings_dessert_binary']]

# Renommer les colonnes pour uniformiser le format des données
usr_rates_dessert.columns = ['recipes_id', 'rates']

# Réinitialiser l'index pour transformer l'index en colonne, comme pour les plats principaux
usr_rates_dessert.reset_index(drop=False, inplace=True)


Création des matrices plat et dessert avec en ligne chaque utilisateur et en colonne chaque recette, faisant apparaître des vecteurs utilisateur/recette

In [11]:
def matrix(usr_rates):
    # Récupère et trie les recettes uniques
    recipes = sorted(set(chain.from_iterable(filter(None, usr_rates['recipes_id']))))
    
    # Liste des utilisateurs
    users = usr_rates['user_id']
    
    # Initialise une matrice de zéros (utilisateurs x recettes)
    M = np.zeros((len(users), len(recipes)))
    
    # Associe chaque recette à un index
    rcp_idx = {recipe: idx for idx, recipe in enumerate(recipes)}
    
    # Remplit la matrice avec les notes des utilisateurs
    for idx, (rcps, rates) in enumerate(zip(usr_rates['recipes_id'], usr_rates['rates'])):
        for rcp, rate in zip(rcps, rates):     
            M[idx, rcp_idx[rcp]] = rate
    
    # Convertit la matrice en DataFrame avec utilisateurs et recettes comme index et colonnes
    df_matrix = pd.DataFrame(M, index=users, columns=recipes)

    return df_matrix

In [12]:
# Construit les matrices utilisateur-recette pour les plats principaux et desserts
df_matrix_main_dish = matrix(usr_rates_main_dish)
df_matrix_dessert = matrix(usr_rates_dessert)

Après observation des données au travers de graphe de réseau, nous avons remarqué la présence d'utilisateur isolé, les utilisateurs isolés sont des utilisateurs ayant noté des recettes que personne d'autre n'a noté, on veut donc les supprimer de nos jeux de donnée.\
Nous avons également pris la décision de retirer :
- Toutes les recettes ayant moins d'une note
- Tous les utilisateurs avec moins de deux notes
- Tous les utilisateurs sans au moins une note positive

Pour la fonction PP(), on boucle jusqu'à ce que les dimensions de nos dataframes ne bouge plus, impliquant que toutes nos règles ci-dessus sont respectés  

In [13]:
def drop_isolated_users(df_matrix):
    # Convertit le DataFrame en matrice NumPy pour des calculs rapides
    np_matrix = df_matrix.to_numpy()
    
    # Compte le nombre d'éléments non nuls par colonne (recettes)
    col_counts = (np_matrix != 0).sum(axis=0)

    # Identifie les colonnes actives (au moins deux utilisateurs ont noté la recette)
    active_cols = col_counts >= 2
    
    # Initialise un masque pour les utilisateurs non isolés
    not_isolated_users = np.zeros(np_matrix.shape[0], dtype=bool)
    
    # Marque comme non isolés les utilisateurs ayant noté une recette active
    for col in np.where(active_cols)[0]:
        not_isolated_users |= (np_matrix[:, col] != 0)
    
    # Identifie les utilisateurs isolés
    isolated_users = df_matrix.index[~not_isolated_users]
    
    # Supprime les utilisateurs isolés du DataFrame
    return df_matrix.drop(index=isolated_users)


In [14]:
def PP(df_matrix):
    # Sauvegarde la forme initiale de la matrice
    shape = df_matrix.shape

    # Filtre initial : Conserve uniquement les utilisateurs ayant au moins 2 notes
    df_filt_matrix = df_matrix.loc[(df_matrix != 0).sum(axis=1) >= 2]

    while True:
        # Étape 1 : Conserve les colonnes (recettes) avec au moins 1 note
        df_filt_matrix = df_filt_matrix.loc[:, (df_filt_matrix != 0).sum(axis=0) >= 1]
        
        # Étape 2 : Conserve les lignes (utilisateurs) avec au moins 2 notes
        df_filt_matrix = df_filt_matrix.loc[(df_filt_matrix != 0).sum(axis=1) >= 2]
        
        # Étape 3 : Conserve les utilisateurs ayant au moins 1 note positive
        df_filt_matrix = df_filt_matrix.loc[(df_filt_matrix == 1).sum(axis=1) >= 1]
        
        # Étape 4 : Supprime les utilisateurs isolés
        df_filt_matrix = drop_isolated_users(df_filt_matrix)

        # Vérifie si la forme de la matrice a changé, sinon termine la boucle
        if df_filt_matrix.shape == shape:
            break
        else:
            shape = df_filt_matrix.shape

    # Retourne la matrice filtrée
    return df_filt_matrix

In [15]:
# Applique le processus de filtrage (PP) pour les plats principaux et desserts
df_filt_matrix_main_dish = PP(df_matrix_main_dish)
df_filt_matrix_dessert = PP(df_matrix_dessert)

À partir de nos matrices, on retourne au format initial, format utilisé par notre algorithme de calcul des distances. 

In [16]:
def matrix_toFormat(df_matrix):
    # Initialise une liste pour stocker les tuples (utilisateur, recette, note)
    lst = []
    
    # Parcourt chaque utilisateur (ligne de la matrice)
    for usr in df_matrix.index:
        # Récupère les recettes et notes de l'utilisateur
        rates = df_matrix.loc[usr][df_matrix.loc[usr] != 0]
        
        # Ajoute un tuple (utilisateur, recette, note) pour chaque recette notée
        for rcp, rate in rates.items():
            lst.append((usr, rcp, rate))
    
    # Convertit la liste en DataFrame avec colonnes 'user_id', 'recipe_id', et 'rate'
    df = pd.DataFrame(lst, columns=['user_id', 'recipe_id', 'rate'])

    # Retourne le DataFrame formaté
    return df

In [17]:
# Convertit les matrices filtrée des plats principaux et desserts en DataFrame formaté
main_dishes = matrix_toFormat(df_filt_matrix_main_dish)
desserts = matrix_toFormat(df_filt_matrix_dessert)

In [18]:
# Sauvegarde les DataFrames des plats principaux et desserts filtrés dans des fichiers CSV
main_dishes.to_csv('data/PP_user_main_dishes.csv')
desserts.to_csv('data/PP_user_desserts.csv')

Dans notre application, on souhaite afficher les noms, descriptions, étapes et ingrédients de chaque recette, On récupère donc tous nos toutes ces infos à partir de nos id finaux  

In [ ]:
# Combine les identifiants de recettes des plats principaux et des desserts
ids = set(main_dishes['recipe_id']).union(set(desserts['recipe_id']))

# Filtre les données des recettes pour ne conserver que celles correspondant aux identifiants sélectionnés
recipes_data = rcp[rcp['id'].isin(ids)].copy()

# Supprime les colonnes inutiles
recipes_data.drop(columns=['minutes', 'contributor_id', 'submitted', 'tags', 'nutrition', 'n_steps', 'n_ingredients', 'type_of_dish'], inplace=True)

# Définir les id des recettes en tant qu'index
recipes_data.set_index('id')

# Réorganise les colonnes
recipes_data = recipes_data[['id', 'name', 'description', 'steps', 'ingredients']]

In [30]:
# Sauvegarde les données filtrées des recettes dans un fichier CSV
recipes_data.to_csv('data/PP_recipes_data.csv')